# [TrackPy walkthrough](http://soft-matter.github.io/trackpy/v0.3.2/tutorial/walkthrough.html#Opening-images-or-video)

First we set up the python environment with the proper plotting and scientific python libraries

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt

# let matplotlib plot inline
%matplotlib inline

# Tweak styles
mpl.rc('figure', figsize=(10,6))
mpl.rc('image',cmap='gray')

import numpy as np
import pandas as pd
# for convenience, pd prefix unnecessary for DataFrame and Series (They're pure pandas)
from pandas import DataFrame, Series

import pims
import trackpy as tp

### Step 1: Read Data
-------

Use [PIMS](http://soft-matter.github.io/pims/v0.3.3/) (Python Image Sequence) to read in data. Supports
* a directory or zipfil of sequential images using `ImageSequence`
* a multi-frame TIFF file using `TiffStack`
* a video (AVI, MOV, ec), using `Video`
* Specialty formats used in microscopy and scientific video capture
 * `Cine`
 * `LSM`
 * Files supported by [Bioformats](https://www.openmicroscopy.org/site/support/bio-formats5.1/supported-formats.html)
 * `ND2` using [PIMS_ND2](https://github.com/soft-matter/pims_nd2)

We'll just use `ImageSequence` for now, others require some extra stuff.

In [4]:
frames = pims.ImageSequence('/media/storage1/Dropbox/LLM_Danny/excitableMembranes/msdAnalysis/AVG_071714ect2-001.xml-C=1/gaussSmooth_2px/*.tif', as_grey=True)

OSError: No files were found matching that path.

In [ ]:
frames

In [ ]:
frames[0]

In [ ]:
plt.imshow(frames[0])

These are basically numpy arrays, but have some extra stuff in them..

In [ ]:
frames[50].frame_no

In [ ]:
frames[50].metadata

### Step 2: Locate Features
-----
Starting with the first frame, estimate the size of the features in pixels (needs to be an odd integer). Using the line tool in ImageJ, I estimate that the dark spots here are 11 pixels (98.43 um/512 pixels = 0.19 um/px). 

Note that the algorithm looks for *bright* spots, so we have to invert the image.

In [ ]:
f = tp.locate(frames[0],11,invert=True)

In [ ]:
f.head()

`locate` returns a `DataFrame` with columns corresponding to:
x position
y position
various characterizations of the objects, will be used to filter
"signal strength" and estimate of uncertainty (derived from paper).

Note that `DataFrames` can easily be exporte to CSV, Excel, SQL, HDF5, etc.

In [ ]:
plt.figure()
tp.annotate(f,frames[0])

#### Refine parameters to eliminate spurious features
Best way to distinguish real particles from spurrious ones is by looking at total brightness ("mass")

In [ ]:
fig,ax = plt.subplots()
ax.hist(f['mass'],bins=20)
ax.set(xlabel='mass',ylabel='count')

From this, we can probably eliminate features with masses less than ~3000. Will also play with other parameter combinations

In [ ]:
f = tp.locate(frames[0],11,invert=True,separation=15, minmass=5000)
plt.figure()
tp.annotate(f,frames[0])

In [ ]:
tp.locate?

A quick way to check for subpixel accuracy is to check that the decimal part of the x and/or y positions are evenly distributed (what?!). If we use a masksize that is too small, we get a dip in the middle

In [ ]:
plt.figure()
tp.subpx_bias(f);

In [ ]:
plt.figure()
tp.subpx_bias(tp.locate(frames[0],7,invert=True,minmass=3500))

Locate features in all frames

In [ ]:
f = tp.batch(frames[:],11,invert=True,separation=15, minmass=5000)

In [ ]:
f.head()

### Step 3: Link features into particle trajectories

Now it's about linking the particles from frame to frame.

- Specify maximum displacement between frames. Pick the smallest reasonable value, larger values slow down computation time
- Set the "memory" the program uses, i.e. the number of frames that an object can skip being identified in but still tracked.

In [ ]:
t=tp.link_df(f,5,memory=3)

In [ ]:
t.head()

#### Filter spurious trajectories

Have some more filtering to do. Ephermeral trajectories, seen for only a few frames, are usually spurious and never really useful. The function `filter_stubs` takes care of these nicely.

In [ ]:
t1 = tp.filter_stubs(t,50)
# compare the number of unfiltered and filtered tracks
print('Before: ', t['particle'].nunique())
print('After: ', t1['particle'].nunique())

In [ ]:
t1.head()

We can also filter trajectories by their appearance. At this stage, with trajectories linked, we can look at a feature's "average appearance" throughout its trajectory, giving a more accurate picture.

In [ ]:
plt.figure()
tp.mass_size(t1.groupby('particle').mean()) # plots size vs. mass

Can filter by cutting off mass and or size regions, for example by using:

`t2 = t1[((t1['mass'] > 250) & (t1['size'] < 3.0) & (t1['ecc'] < 0.1))]`

Seems like we don't have all that many particles, so let's not filter anything and just see what we're left with

In [ ]:
plt.figure()
tp.annotate(t1[t1['frame']==0],frames[0])

Trace trajectories

In [ ]:
plt.figure()
tp.plot_traj(t1)

#### Remove overall drift

Compute the drift and subtract it off

In [ ]:
d = tp.compute_drift(t1)

In [ ]:
plt.figure()
d.plot()

Not really anything here to subtract off, so good!

### Step 4: Analyze the trajectories

#### MSD of individual particles

microns/pixel = 98.43/512

frames per second = 0.2 (taking a guess)

In [ ]:
im = tp.imsd(t1, 98.43/512, 0.2)

In [ ]:
fig, ax = plt.subplots()
ax.plot(im.index, im, 'k-', alpha = 0.1)
ax.set(ylabel = r'$\langle \Delta r ^2 \rangle$ [$\mu$m$^2$]',
      xlabel = 'lag time $t$')
ax.set_xscale('log')
ax.set_yscale('log')

#### Ensemble mean squared displacement

In [ ]:
em = tp.emsd(t1, 98.43/512, 8)

In [ ]:
fig, ax = plt.subplots()
ax.plot(em.index, em, 'o')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $t$')

We can easily fit this to a power law using a convenience function, `fit_powerlaw`, that performs a linear regression in log space.

In [ ]:
plt.figure()
plt.ylabel(r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]')
plt.xlabel('lag time $t$');
tp.utils.fit_powerlaw(em)

# plt.savefig('msd.tif',format='tif')
# plt.savefig('msd.eps', format='eps')

So, we seem to have a power law of very close to one, but we have this interesting oscillating behavior.

This won't really get at the wavelength of the oscillations, but it does seem to give us some temporal information about the oscillations, and the fact that particles seem to move closer to their original position after a time of $\gtrsim 1 s$, while still diffusing away with a fairly close to thermal behavior overall..